In [ ]:
import  os
import  re
import  glob
import  shutil
import  filecmp   
import  sqlparse
import  subprocess
from pathlib    import Path
from itertools  import combinations
from functools  import reduce


FOLDER_DIR  =   "MVPS-BP"
BASE_DIR    =   os.path.join(os.path.abspath(''), FOLDER_DIR)

FOLDER_TO_GIT   =   "typ-entregables-tbp"


! . ./pre-cleaner.sh

In [ ]:
def get_folders(BASE_DIR):
    """
    OBTIENE LOS DIRs Y EVITA TODOS LOS QUE NO CORRESPONDEN PARA EL WALKER
    """
    except_data = ['venv','cleaner_mvp.ipynb','pre-cleaner.sh']
    return sorted([ directory  for directory in glob.glob("*",root_dir=BASE_DIR) if directory not in except_data])


def get_files(BASE_DIR,folder):
    """
    OBTIENE LOS ARCHIVOS PARA PODER INTEGRARLOS A UN ARCHIVO GENERICO INDEXADO
    """
    dir_path = os.path.join(BASE_DIR,folder)
    
    files   =   sorted([ clean_dates_in_sql_files(file,BASE_DIR,folder)  for file in glob.glob('*.sql',root_dir=dir_path) ])
    
    regexp_filtered_files = '.*validacion.*'
    
    
    return [
            file for file in files
            if not re.search( regexp_filtered_files , file,flags=re.IGNORECASE)
    ]


def clean_dates_in_sql_files(f_name,BASE_DIR,folder):
    regexp = "[0-9]{2,4}.[0-9]{2}.[0-9]{2}"
    result  =   re.search(regexp, f_name)
    
    if not result:
        return f_name
    
    pattern =   result.group()
    
    if len(pattern) != 8:
        return f_name
    
    new_name    =   f_name.replace(pattern,f"20{pattern}")
    
    old_path = os.path.join(BASE_DIR,folder,f_name)
    new_path = os.path.join(BASE_DIR,folder,new_name)
    
    os.rename(old_path,new_path)
    
    return new_name

    
    


In [ ]:
def fomartting_sql(file):
    """
    DA UN FORMATO DE SQL USANDO EL PARSE DE UN PACK
    """
    with open(file,'r',encoding='latin1') as f: 
        get_query_to_format = f.read()
           
    return sqlparse.format(get_query_to_format.strip(),keyword_case="upper",indent_tabs=True,reindent=False)



def moving_files(source,destination):
    try:
        
        
        shutil.move(source,destination)
    except Exception as e:
        print(f"""
        Error al mover el archivo   : {source}
        A   : {destination}
    """)
    

def adding_index_file( BASE_DIR, folder ,formatted_files ,output_file):
    __space = "\n"*4
    index_document = ''
    OUTPUT_SAVE_FOLDER = f"{folder}_sql_files"
    
    
    try:
                  
            
        for file in formatted_files:

            header_file = f'-- {file} {__space}'
            index_document += header_file
            
            file__path    =   os.path.join(BASE_DIR,folder,file)
            file__moved__path   =   os.path.join(BASE_DIR,folder,OUTPUT_SAVE_FOLDER,file)
            index_document += fomartting_sql(file__path)
            index_document +=__space
            
            moving_files(file__path,file__moved__path)
                
        with open(output_file,'w',encoding='utf8') as output:
            output.write(index_document)

        return True
    
    except Exception as e:
        print(e)
        
#  tengo problemas con esta cosa


def rename_files(BASE_DIR,folder):
    """
    RENOMBRA CARPETA COMPLICADA
    """
    try:
        regexp_date="[0-9]{4}.[0-9]{2}.[0-9]{2}"
        regexp_step="step\s[0-9]{1,2}"
        
        PATH_FOLDER = os.path.join(BASE_DIR,folder)
        
        
        files = sorted([ file  for file in glob.glob('*.sql',root_dir=PATH_FOLDER) ])

        
        for index,file in enumerate(files):
                        
            match_date =  re.search(regexp_date,file,re.IGNORECASE)
            match_step =  re.search(regexp_step,file,re.IGNORECASE)
            
            old_name = os.path.join(PATH_FOLDER,file)
                
            if match_step:
                a,number_   = match_step.group().split(" ")
                
                if len(number_) == 1:
                    number_ = "0" + number_
                    
                files[index]= f"step_{number_}_{file}"

                if match_date:
                    files[index] = f"{match_date.group()}_{files[index]}"
                    
            new_name = os.path.join(PATH_FOLDER,files[index])
            
            os.rename(old_name,new_name)
    except Exception as e:
        print(e)


def rename_files_clean(BASE_DIR,folder):
    """
    RENOMBRA CARPETA COMPLICADA
    """
    try:
        
        PATH_FOLDER = os.path.join(BASE_DIR,folder)
        
        files = sorted([ file for file in glob.glob('*.sql',root_dir=PATH_FOLDER) ])

        
        for file in files:
            
            new_file_name   =   file.split(" ")[0].replace(".sql", " " +file.split(" ")[1])
            print(new_file_name)
            old_name = os.path.join(PATH_FOLDER,file)
            new_name = os.path.join(PATH_FOLDER,new_file_name)
            
            os.rename(old_name,new_name)

    except Exception as e:
        print(e)


def agregado_commit(folder,FOLDER_DIR):
    
    print(folder)
    
    try:
        regexp_date="[0-9]{4}.[0-9]{2}.[0-9]{2}"
        date = re.search(regexp_date, folder).group()
    
        commit__template = f'cd {FOLDER_DIR} && git add --all "{folder}/" && git commit --date="{date}" -m "Entregable black pumas de la fecha del : {date}" && cd .. '
        result = subprocess.run(commit__template,shell=True , universal_newlines= True, check=True)
        
        print(f">>Commit realizado para la carpeta: {folder}\n>>Con fecha: {date}")
        
        print(result)
    except Exception as e :
        print(f"{e}\nAlgo fallo en el proceso de la carpeta: {folder}")
    


def commit_one_file(folder,FOLDER_DIR,FOLDER_TO_GIT):
    
    print(folder)
    try:
        regexp_date="[0-9]{4}.[0-9]{2}.[0-9]{2}"
        date = re.search(regexp_date, folder).group()
        
        commit__template = f'cp -rv "MVPS-BP/{folder}/"* {FOLDER_TO_GIT} && cd {FOLDER_TO_GIT}  && git add . && git commit --date="{date}" -m "Entregable black pumas de la fecha del : {date}" && cd ..'
        
        result = subprocess.run(commit__template,shell=True , universal_newlines= True, check=True)
        
        print(f">>Commit realizado para la carpeta: {folder}\n>>Con fecha: {date}")
        
        print(result)
    except Exception as e :
        print(f"{e}\nAlgo fallo en el proceso de la carpeta: {folder}")
    
    
def add_file(regexp,files):
    
    return [
        _ for _ in files if re.search(regexp,_)
    ]

def validations_all_permutation(ROOT_DIR, data):
    
    data    =   [ os.path.join(ROOT_DIR,_) for _ in data]
    
    perm    = list(combinations(data,2))
    
    return [
        filecmp.cmp(
            set_data[0],
            set_data[1]
        )
        for set_data in
        perm
    ]
    
def _clean_folder_copies(BASE_DIR,ROOT_DIR):
    
    ROOT_DIR = os.path.join(BASE_DIR,ROOT_DIR)
    
    files = glob.glob("*.sql",root_dir=ROOT_DIR) 
    pasos = sorted(set([re.search("step\s[0-9]{2}",file).group()  for file in files ]))
    pasos_regexp = sorted([f".*{regexp}.*" for regexp in pasos])

    files_to_check  =   {
        paso : add_file( regexp , files)
        for paso,regexp in 
        zip(pasos, pasos_regexp)
    }

    total = []
    validacion  =   lambda a,b: a*b

    for k,v in  files_to_check.items():  
        equals  =   validations_all_permutation(ROOT_DIR,v)
        flag_interno = reduce(validacion, equals)
        
        if not flag_interno:
            print(f"LOS ARCHIVOS PARA ESTOS PASOS NO SON IGUALES: {k}")
        total.append( flag_interno )

    flag    =   reduce(validacion,total)

    if  not flag:
        print("Algun file no es igual")
        
    for copy in range(1,3):
        process = f"cd '{ROOT_DIR}' && find -type f -name '*({copy})*' -delete"
        delete  = subprocess.run(process,shell=True , universal_newlines= True, check=True)
        print(delete)


In [ ]:

def create_sql_files_folder(BASE_DIR,folder):
    try:
        
        sql_folder    =   os.path.join(BASE_DIR,folder,f"{folder}_sql_files")
        os.mkdir(sql_folder)
        return sql_folder
    
    except:
        print(f"Error al crear la subcarpeta para {folder}")
    


def main():
    
    FOLDER = "MVP 2022.11.28"
    FOLDER_WRONG_EXTENSION = "MVP 2022.10.21"    
    ROOT_DIR_MULTIPLE_COPIES    =   "MVP 2023.01.20"
    
    
    print(" >>> RENOMBRADO DE ARCHIVOS")
    rename_files(BASE_DIR,FOLDER)
    print(" >>> LIMPIEZA DE EXTENSIONES INCORRECTAS")
    rename_files_clean(BASE_DIR,FOLDER_WRONG_EXTENSION)
    print(" >>> LIMPIEZA DE DATA DUPLICADA")
    _clean_folder_copies(BASE_DIR,ROOT_DIR_MULTIPLE_COPIES)
    
    
    
    folders     =   get_folders(BASE_DIR)
    len_folder  =   len(folders)
    
    for index,folder in enumerate(folders):
        print(f""" 
        PROCESO {index+1} DE {len_folder}
        GENERANDO ARCHIVO PARA EL FOLDER {folder}
        """)
        
        sql_file_store    =   create_sql_files_folder(BASE_DIR, folder)
        
        
        try:
            files = get_files(BASE_DIR ,folder)
            folder_path =   os.path.join(BASE_DIR,folder)
            output_path =   os.path.join(BASE_DIR,folder, "mvp_bp_index.sql")
            path_all_files  =   [os.path.join(BASE_DIR,folder,__path) for __path in files]
            
            adding_index_file(BASE_DIR, folder, files , output_path )
            
            print(f"""
                  >>> EL PROCESO PARA ESTA CARPETA CORRIO SIN PROBLEMAS
                  """)
            
            
            
    #         # agregado_commit(folder,FOLDER_DIR)
    
            commit_one_file(folder,FOLDER_DIR,FOLDER_TO_GIT)
    
        
        except :
            print(f"""FALLO ALGO EN EL PROCESO DE LA CARPETA: {folder}""")

    
        

In [ ]:
# MVP 2023.01.20J
# MVP 2023.02.17

main()